In [9]:
pip install s3fs

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
sft_trainer_config = '''{
    "model_name_or_path": "meta-llama/Llama-3.1-8B-Instruct",
    "training_data_path": "epfl-dlab/gsm8k",
    "validation_data_path": "epfl-dlab/gsm8k",
    "output_dir": "/mnt/output/model",
    "save_model_dir": "/mnt/output/model",
    "num_train_epochs": 3.0,
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 4,
    "gradient_accumulation_steps": 4,
    "gradient_checkpointing": true,
    "save_strategy": "epoch",
    "learning_rate": 1e-5,
    "weight_decay": 0.0,
    "lr_scheduler_type": "cosine",
    "include_tokens_per_second": true,
    "data_formatter_template": "### Question:{{question}} \\n\\n### Answer: {{answer}}",
    "response_template": "\\n### Answer:",
    "use_flash_attn": true,
    "fast_kernels": [true, true, true],
    "peft_method": "lora",
    "target_modules": ["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    "lora_post_process_for_vllm": true
}'''

In [16]:
import base64

def encode_config(config):
    base64_bytes = base64.b64encode(config.encode("ascii"))
    txt = base64_bytes.decode("ascii")
    return txt

encoded_config = encode_config(sft_trainer_config)

In [17]:
from kubeflow.training import TrainingClient
from kubernetes import client
from kubernetes.client import (
    V1EnvVar,
    V1EnvVarSource,
    V1SecretKeySelector,
    V1Volume,
    V1VolumeMount,
    V1PersistentVolumeClaimVolumeSource
)

job_name = "llama4"

tc = TrainingClient()

tc.create_job(
    job_kind="PyTorchJob",
    name=job_name,
    num_workers=1,
    num_procs_per_worker="auto",
    resources_per_worker={"gpu": 1},
    base_image="quay.io/modh/fms-hf-tuning:v2.8.2",
    env_vars=[
        V1EnvVar(name="HF_TOKEN", value_from=V1EnvVarSource(secret_key_ref=V1SecretKeySelector(key="HF_TOKEN", name="hf-token"))),
        V1EnvVar(name="SFT_TRAINER_CONFIG_JSON_ENV_VAR", value=encoded_config),
    ],
    volumes=[
        V1Volume(name="llama70",
                 persistent_volume_claim=V1PersistentVolumeClaimVolumeSource(claim_name="llama70")),
        V1Volume(name="trained-model",
                 persistent_volume_claim=V1PersistentVolumeClaimVolumeSource(claim_name="trained-model")),
    ],
    volume_mounts=[
        V1VolumeMount(name="llama70", mount_path="/mnt/model"),
        V1VolumeMount(name="trained-model", mount_path="/mnt/output"),
    ],
)

In [8]:
logs, _ = tc.get_job_logs(job_name, follow=True)

[Pod llama4-master-0]: WARNING:accelerate.commands.launch:The following values were not passed to `accelerate launch` and had defaults used instead:
[Pod llama4-master-0]: 	`--num_machines` was set to a value of `1`
[Pod llama4-master-0]: 	`--mixed_precision` was set to a value of `'no'`
[Pod llama4-master-0]: 	`--dynamo_backend` was set to a value of `'no'`
[Pod llama4-master-0]: To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[Pod llama4-master-0]: /home/tuning/.local/lib/python3.12/site-packages/fms_acceleration_moe/utils/checkpoint_utils.py:367: SyntaxWarning: invalid escape sequence '\.'
[Pod llama4-master-0]:   _reg = re.compile(f"(.*)\.({_name})\.weight")
[Pod llama4-master-0]: /home/tuning/.local/lib/python3.12/site-packages/fms_acceleration_moe/utils/checkpoint_utils.py:367: SyntaxWarning: invalid escape sequence '\.'
[Pod llama4-master-0]:   _reg = re.compile(f"(.*)\.({_name})\.weight")
[Pod llama4-master-0]: /home/tunin

In [10]:
import s3fs
import os


s3 = s3fs.S3FileSystem(
      key=os.environ["AWS_ACCESS_KEY_ID"],
      secret=os.environ["AWS_SECRET_ACCESS_KEY"],
      endpoint_url=os.environ["AWS_S3_ENDPOINT"]
   )
s3_path = os.environ["AWS_S3_BUCKET"] + "/meta-llama/Llama-3.1-8B-Instruct/adapter"
_ = s3.put("/opt/app-root/src/model", s3_path, recursive=True)

In [12]:
pip install openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.4/680.4 kB 84.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [45]:
import os
import httpx
from openai import OpenAI


client = OpenAI(
    http_client = httpx.Client(verify=False) 
)

completion = client.chat.completions.create(
  model="gsm8k",
  messages=[
    {"role": "user", "content": "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"}
  ]
)

print(completion.choices[0].message.content)

Janet eats 3 eggs for breakfast, and bakes 4 eggs into muffins, so she uses 3 + 4 = 7 eggs.

She lays 16 eggs per day, and uses 7, so she has 16 - 7 = 9 eggs left.

She sells 9 eggs at $2 each, so she makes 9 * 2 = $<<9*2=18>>18 per day at the farmers' market.


In [1]:
pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 239.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 275.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 144.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 227.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 227.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 634.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 323.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 975.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 331.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 285.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 225.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 321

In [5]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer


base_model_name = "meta-llama/Llama-3.1-8B-Instruct"
adapter_model_name = "/opt/app-root/src/model/checkpoint-1401/"

base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
model = PeftModel.from_pretrained(base_model, adapter_model_name)
model = model.merge_and_unload()
# model.save_pretrained("merged_adapters")

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
base_model_merged = AutoModelForCausalLM.from_pretrained("/opt/app-root/src/fms-hf-tuning-example/merged_adapters")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
import transformers
import torch


# Test the fine-tuned model
pipeline = transformers.pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {
        "role": "user",
        "content": "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
    }
]

outputs = pipeline(messages, max_new_tokens=256, temperature = 0.01)

output = ""
for turn in outputs:
    for item in turn["generated_text"]:
        output += f"# {item['role']}\n\n{item['content']}\n\n"

display(output)

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"# user\n\nJanet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\n\n# assistant\n\nJanet eats 3 eggs for breakfast and bakes 4 for muffins, so she uses 3 + 4 = <<3+4=7>>7 eggs.\nShe lays 16 eggs per day and uses 7, so she has 16 - 7 = <<16-7=9>>9 eggs left.\nShe sells the 9 eggs at $2 each, so she makes 9 * 2 = $<<9*2=18>>18 per day at the farmers' market.\n\n"

In [7]:
import gc

del base_model 
del model

gc.collect()
torch.cuda.empty_cache()

In [8]:
import s3fs
import os


s3 = s3fs.S3FileSystem(
      key=os.environ["AWS_ACCESS_KEY_ID"],
      secret=os.environ["AWS_SECRET_ACCESS_KEY"],
      endpoint_url=os.environ["AWS_S3_ENDPOINT"]
   )
s3_path = os.environ["AWS_S3_BUCKET"] + "/meta-llama/Llama-3.1-8B-Instruct/adapter"
s3.get(s3_path, ".", recursive=True)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]